In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import pandas as pd

import nltk
nltk.download('punkt_tab')

from nltk.corpus import stopwords
nltk.download('stopwords')


import re
from unicodedata import normalize as norm



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def text_cleaner(text):

    nltk_stopwords = stopwords.words('english')

    collection_text = [ {"text" : text}]
    text = pd.DataFrame(collection_text)

    text['text'] = text['text'].astype('str')
    text['text'] = text['text'].str.lower()
    text['text'] = text['text'].str.replace('\n',' ')
    text['text'] = text['text'].str.replace('\r',' ')
    text['text'] = text['text'].apply(lambda x: norm('NFKD', x).encode('ascii', 'ignore').decode())
    text['text'] = text['text'].apply(lambda x: re.sub(r'[^a-zA-Z]',' ',x))
    text['text'] = text['text'].apply(lambda x: re.sub(r'\s+',' ',x))
    pat = r'\b(?:{})\b'.format('|'.join(nltk_stopwords))
    text['text'] = text['text'].str.replace(pat,'')
    text = text['text'].values[0]

    return text

In [ ]:
# load dataset

In [ ]:
df = pd.read_csv("posts_final.csv")

In [ ]:
df.head()

,ID,Date,Subreddit,URL,Title,Text,Author,Score,num_comment
0,1gl2h1y,2024-11-06 16:28:01,womenintech,https://www.reddit.com/r/womenintech/comments/...,Election talk at work: don’t take the bait,"If anyone, but especially men, at work want to...",user_1,10676,484
1,1hcv1ab,2024-12-12 20:51:55,womenintech,https://www.reddit.com/r/womenintech/comments/...,Stand your ground. You’re not a secretary,What are the creative ways you’ve had to tell ...,user_2,5001,396
2,1fzvk1s,2024-10-09 16:28:10,womenintech,https://fortune.com/2024/08/08/most-high-perfo...,Around 76% of high-performing women receive ne...,About 76% of top-performing working women rece...,user_3,4278,327
3,1eemu9y,2024-07-29 01:01:50,womenintech,https://www.reddit.com/r/womenintech/comments/...,Colleague assumed I was pregnant when I said “...,"A few weeks ago, I was taking a virtual meetin...",user_4,2698,148
4,1ey5rsx,2024-08-22 00:32:53,womenintech,https://www.reddit.com/r/womenintech/comments/...,This is why women leave tech,I feel like I'm at a breaking point here. I lo...,user_5,2146,197


In [ ]:
df['Post']= df['Title']+" "+df['Text']

In [ ]:
# clean post

In [ ]:
df["PostClean"] = df["Post"].apply(lambda x: text_cleaner(x))

## Config Bert model

In [ ]:
def classify(text:str):
  encoded_input = tokenizer(text[:511], return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  ranking = np.argsort(scores)
  ranking = ranking[::-1]

  return labels[ranking[0]]

In [ ]:
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

taskes=['emotion', 'hate', 'irony', 'offensive', 'sentiment']

In [ ]:
for task in taskes:
  MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

  tokenizer = AutoTokenizer.from_pretrained(MODEL)

  labels=[]
  mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
  with urllib.request.urlopen(mapping_link) as f:
      html = f.read().decode('utf-8').split("\n")
      csvreader = csv.reader(html, delimiter='\t')
  labels = [row[1] for row in csvreader if len(row) > 1]

  #PT
  model = AutoModelForSequenceClassification.from_pretrained(MODEL)
  model.save_pretrained(MODEL)
  df[task] = df['PostClean'].apply(lambda x: classify(x))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
df.head()

,ID,Date,Subreddit,URL,Title,Text,Author,Score,num_comment,Post,PostClean,emoji,emotion,hate,irony,offensive,sentiment
0,1gl2h1y,2024-11-06 16:28:01,womenintech,https://www.reddit.com/r/womenintech/comments/...,Election talk at work: don’t take the bait,"If anyone, but especially men, at work want to...",user_1,10676,484,Election talk at work: don’t take the bait If...,election talk at work dont take the bait if an...,😂,anger,not-hate,non_irony,not-offensive,negative
1,1hcv1ab,2024-12-12 20:51:55,womenintech,https://www.reddit.com/r/womenintech/comments/...,Stand your ground. You’re not a secretary,What are the creative ways you’ve had to tell ...,user_2,5001,396,Stand your ground. You’re not a secretary Wha...,stand your ground youre not a secretary what a...,😂,anger,not-hate,non_irony,not-offensive,neutral
2,1fzvk1s,2024-10-09 16:28:10,womenintech,https://fortune.com/2024/08/08/most-high-perfo...,Around 76% of high-performing women receive ne...,About 76% of top-performing working women rece...,user_3,4278,327,Around 76% of high-performing women receive ne...,around of high performing women receive negati...,✨,sadness,not-hate,irony,not-offensive,negative
3,1eemu9y,2024-07-29 01:01:50,womenintech,https://www.reddit.com/r/womenintech/comments/...,Colleague assumed I was pregnant when I said “...,"A few weeks ago, I was taking a virtual meetin...",user_4,2698,148,Colleague assumed I was pregnant when I said “...,colleague assumed i was pregnant when i said i...,😂,joy,not-hate,non_irony,not-offensive,negative
4,1ey5rsx,2024-08-22 00:32:53,womenintech,https://www.reddit.com/r/womenintech/comments/...,This is why women leave tech,I feel like I'm at a breaking point here. I lo...,user_5,2146,197,This is why women leave tech I feel like I'm a...,this is why women leave tech i feel like i m a...,😂,anger,not-hate,irony,not-offensive,positive


In [ ]:
df.to_csv("post_label.csv")